In [22]:
!pip install -qU langchain-openai


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [23]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain

In [2]:
# Add prereq code to set open API key
# Load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv()

import os
os.environ["OPENAI_API_KEY"] = os.environ.get('API_KEY')
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

First, initialize your LLM

In [24]:
# TODO: initialize OpenAI object with your API key
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm.openai_api_key = os.environ["OPENAI_API_KEY"] 
llm.openai_api_base = os.environ["OPENAI_API_BASE"]

Then, load reviews from tv-reviews.csv

In [25]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(file_path='./tv-reviews.csv',
    csv_args={
    'delimiter': ',',
    'quotechar': '"',
    'fieldnames': ['TV Name', 'Review Title', 'Review Rating', 'Review Text']
})

docs = loader.load()
print(docs)

[Document(metadata={'source': './tv-reviews.csv', 'row': 0}, page_content='TV Name: TV Name\nReview Title: Review Title\nReview Rating: Review Rating\nReview Text: Review Text'), Document(metadata={'source': './tv-reviews.csv', 'row': 1}, page_content="TV Name: Imagix Pro\nReview Title: Amazing Picture Quality\nReview Rating: 9\nReview Text: I recently purchased the Imagix Pro and I am blown away by its picture quality. The colors are vibrant and the images are crystal clear. It feels like I'm watching movies in a theater! The sound is also impressive, creating a truly immersive experience. Highly recommended!"), Document(metadata={'source': './tv-reviews.csv', 'row': 2}, page_content="TV Name: Imagix Pro\nReview Title: Impressive Features\nReview Rating: 8\nReview Text: The Imagix Pro is packed with impressive features that enhance my viewing experience. The smart functionality allows me to easily stream my favorite shows and movies. The remote control is user-friendly and has conveni

Split the documents you loaded into smaller chunks

In [26]:
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

Now, initialize your embeddings

In [27]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
embeddings.openai_api_key = os.environ["OPENAI_API_KEY"]
embeddings.openai_api_base = os.environ["OPENAI_API_BASE"]
db = Chroma.from_documents(split_docs, embedding=embeddings)

Initialize your vector db with your embeddings model and populate with your text chunks

Query your vector database for 5 most semantically similar chunks

In [28]:
query = """
    Based on the reviews in the context, tell me what people liked about the picture quality.
    Make sure you do not paraphrase the reviews, and only use the information provided in the reviews.
    """

similar_docs = db.similarity_search(query, k=5)
print(len(similar_docs))

5


Combined, they should provide enough information to answer our question about picture quality

In [29]:
prompt = PromptTemplate(
    template="{query}\nContext: {context}",
    input_variables=["query", "context"],
)
chain = load_qa_chain(llm, prompt = prompt, chain_type="stuff")
print(chain.run(input_documents=similar_docs, query=query))


People liked the vibrant colors and crystal clear images of the picture quality on the Imagix Pro TV. They also appreciated the sharpness and lifelike details that made the viewing experience feel immersive and like watching movies in a theater.
